# Building a Chatbot using Gemini API

#### Task 0: Configure API key

The Python SDK for the Gemini API is contained in the google-generativeai package. 

Install dependency using:
*pip install -q -U google-generativeai
*

Do not check an API key into your version control system but assign it as an environment variable instead:
export API_KEY=<YOUR_API_KEY>

#### Task 1: Initialize the model

In [ ]:
# !pip install google-generativeai
import google.generativeai as genai

with open('gemini_api_key.txt','r') as file:
    API_KEY = file.read()
genai.configure(api_key=API_KEY)

Use system instructions to steer the behavior of a model

In [ ]:
# Using a context manager to open the file
with open('pizzabot_system_instruction.txt', 'r') as file:
    sys_instr = file.read()

In [ ]:
model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=sys_instr)

##### 2.1 Initialize the chat
chat = model.start_chat(history=[])

#### Task 2: Receive Prompts and Save Context and Generate chat responses
*I searched for gemini api help docs.* 

**ChatSession** class of gemini enables us to have freeform conversation over multiple turns. We dont have to store conversation history as a list.

Example: [Build an interactive chat](https://ai.google.dev/gemini-api/docs/text-generation?lang=python#chat)  

The **ChatSession.send_message** method returns the same *GenerateContentResponse* type as **GenerativeModel.generate_content**. It also appends your message and the response to the chat history

#### Outline of chat using gemini api
```
model=genai.GenerativeModel( \
  model_name="gemini-1.5-flash") \
chat = model.start_chat( \
    history=[ \
        {"role": "user", "parts": "Hello"}, 
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)
response = chat.send_message("I have 2 dogs in my house.")
print(response.text)
response = chat.send_message("How many paws are in my house?")
print(response.text)
```

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = chat.send_message(context) 
    context.append({'role':'model', 'parts':f"{response}"})
    panels.append(
        pn.Row('user:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('model:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [ ]:
#### Task 3: Build GUI 
import panel as pn
pn.extension() #initialization

panels = [] # collect display

In [ ]:
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")
interactive_conversation = pn.bind(collect_messages, button_conversation)
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)
dashboard

In [ ]:
def run_chat(value):
    if not value:
        value = chat.history
    response = chat.send_message(value)
    if response:
        return f"Submitted: {response.text}"
    else: 
        return f"Submitted: "
    

In [ ]:
async def gemini_callback(contents, user, instance):
    # response = await openai.ChatCompletion.acreate(
    #     model="gpt-3.5-turbo",
    #     messages=[{"role": "user", "content": contents}],
    #     stream=True,
    # )
    response = await chat.send_message(contents)
    
    message = ""
    async for chunk in response:
        message += chunk["choices"][0]["delta"].get("content", "")
        yield message


In [ ]:
def run_chat(value):
    print(chat)
    print(value)
    response = chat.send_message(value)
    return f"Assitant: {response.text}"

In [26]:
def output(value):
    print(chat)
    response = chat.send_message(value)
    return f"Assitant: {response.text}"

chat_area_input = pn.chat.ChatAreaInput(value="Hi",placeholder="Type something, and press Enter to submit!")
output_markdown = pn.bind(run_chat, chat_area_input.param.value)
pn.Column(chat_area_input, output_markdown)

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-flash',
        generation_config={},
        safety_settings={},
        tools=None,
        system_instruction="You are OrderBot, an automated service to collect orders for a pizza restaurant. \nYou first greet the customer, then collects the order,\nand then asks if it's a pickup or delivery.\nYou wait to collect the entire order, then summarize it and check for a final \ntime if the customer wants to add anything else. \nIf it's a delivery, you ask for an address. \nFinally you collect the payment.\nMake sure to clarify all options, extras and sizes to uniquely \nidentify the item from the menu.\nYou respond in a short, very conversational friendly style. \nThe menu includes \npepperoni pizza  12.95, 10.00, 7.00 \ncheese pizza   10.95, 9.25, 6.50 \neggplant pizza   11.95, 9.75, 6.75 \nfries 4.50, 3.50 \ngreek salad 7.25 \nToppings: \nextra cheese 2.00, \nmushrooms 1.50 \nsausage 3.00 \ncanadian baco

BokehModel(combine_events=True, render_bundle={'docs_json': {'3c8e4a6f-5879-4dd3-87d3-345f39ce616f': {'version…

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-flash',
        generation_config={},
        safety_settings={},
        tools=None,
        system_instruction="You are OrderBot, an automated service to collect orders for a pizza restaurant. \nYou first greet the customer, then collects the order,\nand then asks if it's a pickup or delivery.\nYou wait to collect the entire order, then summarize it and check for a final \ntime if the customer wants to add anything else. \nIf it's a delivery, you ask for an address. \nFinally you collect the payment.\nMake sure to clarify all options, extras and sizes to uniquely \nidentify the item from the menu.\nYou respond in a short, very conversational friendly style. \nThe menu includes \npepperoni pizza  12.95, 10.00, 7.00 \ncheese pizza   10.95, 9.25, 6.50 \neggplant pizza   11.95, 9.75, 6.75 \nfries 4.50, 3.50 \ngreek salad 7.25 \nToppings: \nextra cheese 2.00, \nmushrooms 1.50 \nsausage 3.00 \ncanadian baco